In [1]:
import pandas as pd
import numpy as np
import random
import datetime
import boto3
import botocore
import psycopg2
import configparser

### Nombre de la instancia

In [2]:
rdsIdentifier = 'proyecto-aws-v1'


### Cargar archivo de config.

In [3]:
config = configparser.ConfigParser()
config.read('proyecto.cfg')

['proyecto.cfg']

### Crear instancia

In [5]:
aws_data = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                     aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                     region_name='us-east-1')

### Instancias disponibles

In [6]:
rdsInstanceIds = []

response = aws_data.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['banco-db', 'dw-db', 'proyecto-aws-v1']


### Creacion de servicio RDS

In [7]:
try:
    response = aws_data.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="postgres",
            MasterUsername=config.get('RDS', 'DB_USER'),
            MasterUserPassword=config.get('RDS', 'DB_PASSWORD'),
            Port=int(config.get('RDS', 'DB_PORT')),
            VpcSecurityGroupIds=["sg-02abf94b67ee35a24"],
            PubliclyAccessible=True
        )
    print (response)
except aws_data.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.")

La Instancia de Base de Datos ya Existe.


### Obteniendo URL del host

In [13]:
try:
     instances = aws_data.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)

proyecto-aws-v1.cznwk1uf05mh.us-east-1.rds.amazonaws.com


### Conexión a base de datos

In [21]:
import sql_queries_AWS_V2

try:
    db_conn = psycopg2.connect(
        database=config.get('RDS', 'DB_NAME'), 
        user=config.get('RDS', 'DB_USER'),
        password=config.get('RDS', 'DB_PASSWORD'), 
        host=RDS_HOST,
        port=config.get('RDS', 'DB_PORT')
    )

    cursor = db_conn.cursor()
    cursor.execute(sql_queries_AWS_V2.DDL_QUERY)
    db_conn.commit()
    print("Base de Datos Creada Exitosamente")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)

ERROR: Error al crear la base de datos.
syntax error at or near "AUTO_INCREMENT"
LINE 5:   Adquisicion_FK int NOT NULL AUTO_INCREMENT,
                                      ^



### Función para insertar datos

In [ ]:
def insertDataToSQL(data_dict, table_name):
     postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""    
     df_data = pd.DataFrame.from_records(data_dict)
     try:
          response = df_data.to_sql(table_name, postgres_driver, index=False, if_exists='append')
          print(f'Se han insertado {response} nuevos registros.' )
     except Exception as ex:
          print(ex)

### Insertar datos de aqui en adelante

### Insertar datos para evento

### Insertar precios